# Contents <a id='back'></a>

* [Introduction](#intro)
* [1. Data Overview](#data_review)
    * [Conclusions](#data_review_conclusions)
* [2. Data Pre-Processing](#data_preprocessing)
    * [2.1 Drop Unused Columns](#drop_unused_cols)
* [3. Splitting Data and Training Model](#splitting_training)
* [4. Profit Calculation](#profit_calculation)
    * [4.1 Preparing Parameter](#parameter)
    * [4.2 Profit Calculation in top 200 wells](#top200_profit)
    * [4.3 Comparison of Profit Values based on prediction data and real data](#predict_real_comparison)
* [5. Calculation of Risk and Profit Using Bootstrapping Technique](#bootstrap_profit)
    * [5.1 Function to Calculate Profit](#bootstrap_function)
    * [5.2 Calculating profit in prediction data](#bootstrap_prediction)
    * [5.3 Calculating profit in real data](#bootstrap_real)
    * [5.4 Comparison of Profit Values based on prediction data and real data using bootstrap technique](#bootstrap_predict_real_comparison)
* [General Conclusion](#end)

# Introduction <a id='intro'></a>

In this project, I will train a model that can predict suitable locations for drilling new oil wells. The data used includes sample oil data from three regions. The parameters of each oil in these regions are already known.

**Objective:**

The goal of this project is to recommend a region for oil well development along with justification or reasoning for the region's selection. In this process, bootstrapping will be performed with 1,000 samples to discover the profit distribution. Additionally, profit and risk calculations will be carried out for each region.

**This project will comprise the following steps:**

1. Data Overview
2. Splitting the Data
3. Assessing Model Quality
4. Hyperparameter Tuning

[Back to Contents](#back)

## 1. Data Overview <a id='data_review'></a>

The steps to be performed are as follows:
1. Checking the number of rows and columns.
2. Checking for missing values.
3. Checking for duplicate data.
4. Checking statistical information in columns with numerical data types.
5. Checking values in columns with categorical data types.

[Back to Contents](#back)

In [1]:
# load library

# dataset
import pandas as pd, numpy as np

# scientific computing
import numpy as np

from numpy.random import RandomState

# model
from sklearn.linear_model import LinearRegression

# splitting data
from sklearn.model_selection import train_test_split

# shuffle array
from sklearn.utils import shuffle

# calculating mse
from sklearn.metrics import mean_squared_error

# a progress bar interface for loops and operations that take some time to complete
from tqdm import tqdm

# ignore warning
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings("ignore")

In [7]:
# path dataset
path_df_region0 = 'data/geo_data_0.csv'
path_df_region1 = 'data/geo_data_1.csv'
path_df_region2 = 'data/geo_data_2.csv'

In [8]:
# load dataset
df0 = pd.read_csv(path_df_region0)
df1 = pd.read_csv(path_df_region1)
df2 = pd.read_csv(path_df_region2)

### 1.1 Data Exploration: region0

In [9]:
df0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [12]:
df0.shape

(100000, 5)

In [13]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [14]:
# check duplicated data
df0.isnull().sum() / 100

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

In [15]:
df0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


### 1.2 Data Exploration: region1

In [10]:
df1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [16]:
df1.shape

(100000, 5)

In [17]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [18]:
# check duplicated data
df1.isnull().sum() / 100

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

In [19]:
df1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


### Data Exploration: region2

In [11]:
df2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [21]:
df2.shape

(100000, 5)

In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [23]:
# check duplicated data
df2.isnull().sum() / 100

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

In [24]:
df2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


### Conclusion <a id='data_review_conclusions'></a>

**df0**

1. There are no missing values.
2. The data types in the columns are correct.
3. The 'id' column will be dropped as it is not used.

**df1**
1. There are no missing values.
2. The data types in the columns are correct.
3. The 'id' column will be dropped as it is not used.

**df2**
1. There are no missing values.
2. The data types in the columns are correct.
3. The 'id' column will be dropped as it is not used.

## 2. Data Preprocessing <a id='data_preprocessing'></a>

[Back to Contents](#back)

### 2.1 Drop Unused Columns <a id='drop_unused_cols'></a>

In [25]:
all_data = [
    df0.drop('id', axis =1),
    df1.drop('id', axis =1),
    df2.drop('id', axis =1),
]

## 3. Splitting Data and Training Model <a id='splitting_training'></a>

[Back to Contents](#back)

In [26]:
state = RandomState(42)

samples_target = []
samples_prediction = []

for region in range(len(all_data)):
    data = all_data[region]
    
    features = data.drop('product', axis = 1)
    target = data['product']
    # split training and testing
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state = state)
    
    # training model
    # model Linear Regression
    model = LinearRegression()
    model.fit(features_train, target_train)
    # predict using training set
    predictions = model.predict(features_valid)
    
    samples_target.append(target_valid.reset_index(drop=True))
    samples_prediction.append(pd.Series(predictions))

    # calculate root mean squared error on training dataset
    rmse_train = np.sqrt(mean_squared_error(target_valid, predictions))
    
    # Average Barrel
    avg_product_target = target.mean()
    avg_product_predictions = predictions.mean()
    
    # print the result
    print('--------------------------------')
    print(f'Training Result Geo Data {region}')
    print(f'Average Barrel Target: {avg_product_target}')
    print(f'Average Barrel Predictions: {avg_product_predictions}')
    print(f'RMSE Training: {rmse_train}')

--------------------------------
Training Result Geo Data 0
Average Barrel Target: 92.50000000000001
Average Barrel Predictions: 92.3987999065777
RMSE Training: 37.756600350261685
--------------------------------
Training Result Geo Data 1
Average Barrel Target: 68.82500000000002
Average Barrel Predictions: 68.58780312701154
RMSE Training: 0.8899472027156135
--------------------------------
Training Result Geo Data 2
Average Barrel Target: 95.00000000000004
Average Barrel Predictions: 95.09230341410031
RMSE Training: 40.23735038140562


**Analysis Results:**

1. Based on the average product reserves, Region 2 has the highest value.
2. Looking at the RMSE values, it can be observed that df1 has the smallest RMSE (Root Mean Square Error) value. This indicates that the exact average barrel value lies within the range of 67.81 (68.72 - 0.89) to 69.7 (68.72 + 0.89).

## 4. Profit Calculation <a id='profit_calculation'></a>

[Back to Contents](#back)

### 4.1 preparing the parameter for calculating the profit <a id='parameter'></a>

In [27]:
product_price = 4500
total_cost = 100000000 # 100 juta USD
points = 200
cost_per_point = total_cost/points
point_per_cost = int(total_cost / cost_per_point)
point_per_cost

200

### 4.2 Profit Calulation in top 200 wells <a id='top200_profit'></a>

**The process to be conducted in this stage is as follows:**

Creation of functions to determine:
- The highest profit from the three regions
- Total cost and total income
- Total oil reserves
- Wells with the highest predicted values

In the creation of these functions, the following considerations will be taken into account:

a. Only the top 200 wells will be selected.

b. The budget for developing 200 oil wells is 100 million USD (total_cost).

c. Income is 4500 per 1000 barrels (income).

Subsequently, the target (actual) volume of oil reserves will be calculated based on the predictions that have been made.

#### 4.2.1 Prediction Data

In [28]:
# function to calculae profit in top 200 wells in prediction data
def profitability(prediction, name, income=4500, total_cost = 100000000, points = 200):
    prediction = prediction[name]
    prediction_top200 = prediction.sort_values(ascending=False)[:points].reset_index()
    total_oil = prediction_top200[0].sum()
    total_cost = round(total_cost/1000000)
    total_income = round(income * total_oil/1000000)
    profit = round(total_income - total_cost)
    
    # print the result
    print('-------------------------------')
    print(f'Profitability Geo Data: {name}')
    print(f'Profit: {profit}', 'M USD')
    print(f'Total Cost: {total_cost}', 'M USD')
    print(f'Total Income: {total_income}', 'M USD')
    print(f'Total oil reserves: {total_oil}')

In [29]:
# apply the function to know the profit and oil reserves
profitability(prediction=samples_prediction, name = 0)
profitability(prediction=samples_prediction, name = 1)
profitability(prediction=samples_prediction, name = 2)

-------------------------------
Profitability Geo Data: 0
Profit: 39 M USD
Total Cost: 100 M USD
Total Income: 139 M USD
Total oil reserves: 30881.463288146995
-------------------------------
Profitability Geo Data: 1
Profit: 25 M USD
Total Cost: 100 M USD
Total Income: 125 M USD
Total oil reserves: 27746.394260320503
-------------------------------
Profitability Geo Data: 2
Profit: 35 M USD
Total Cost: 100 M USD
Total Income: 135 M USD
Total oil reserves: 29941.147885588958


**Conclusion**

1. From the results of the created functions, it can be observed that Geo Data Region 0 has the highest profit, highest total income, highest total oil reserves, and the highest volume of oil reserves in a well. 
2. However, these results are not final, as they need to consider the RMSE values from the previous process, where Geo Data Region 0 had a high risk of 37%.

#### 4.2.2 Real Data

In [30]:
top200_predictions = samples_prediction[0].sort_values(ascending=False)[:point_per_cost]

In [31]:
top200_predictions

6958     176.536104
18194    176.274510
17251    173.249504
457      172.802708
2202     172.744977
            ...    
8772     146.952653
15975    146.948764
7846     146.947483
6633     146.946515
7154     146.874947
Length: 200, dtype: float64

In [32]:
# Matching the indices in the prediction results with the data in the `samples_target` dataset.
top200_target = samples_target[0][top200_predictions.index]
top200_target

6958     153.639837
18194    140.631646
17251    178.879516
457      176.807828
2202     130.985681
            ...    
8772     132.951877
15975    136.027691
7846     162.142530
6633     120.536962
7154     138.424174
Name: product, Length: 200, dtype: float64

In [33]:
# calculating total product or total oil
total_product = top200_target.sum()
total_product

29686.9802543604

In [35]:
# calculating revenue
revenue = total_product*4500
revenue

133591411.14462179

In [36]:
# calculating profit
profit = revenue - total_cost
profit

33591411.14462179

**From the above steps of calculating real profit, a function is created to calculate profit, as shown below:**

In [38]:
# function to calculate profit

def target_profit(target, predictions):
    predictions_sorted = predictions.sort_values(ascending=False)
    selected_points = target[predictions_sorted.index][:point_per_cost]
    product = selected_points.sum()
    revenue = product * product_price
    profit = revenue - total_cost
    return profit

In [39]:
# apply the function in each region
print('Real profit based on prediction results in region 0:',target_profit(samples_target[0], samples_prediction[0]))
print('Real profit based on prediction results in region 1:',target_profit(samples_target[1], samples_prediction[1]))
print('Real profit based on prediction results in region 2:',target_profit(samples_target[2], samples_prediction[2]))

Real profit based on prediction results in region 0: 33591411.14462179
Real profit based on prediction results in region 1: 24150866.966815114
Real profit based on prediction results in region 2: 25841568.87152408


**Conclusion**

1. From the results of the created function, it can be observed that Geo Data Region 0 has the highest real profit, amounting to 34 million USD.
2. These results are not final, as the high risk of 37% in Geo Data Region 0 must still be considered based on the RMSE value from the previous process.

### 4.3 Comparison of Profit Values between Predicted Data and Real Data <a id='predict_real_comparison'></a>

In the predicted data, the profit value for region 0 is 40M USD. However, when using the real data (samples_target), the profit value for region 0 is 33M USD. Therefore, there is an error of approximately 7M in the predicted data.

## 5. Calculation of Risk and Profit Using Bootstrapping Technique <a id='boostrap_technique'></a>

**Conditions**

1. In utilizing the bootstrapping technique, a sample of the top 500 will be taken, with 1,000 iterations performed.
2. From the bootstrapping results, the top 200 (best 200 points) will be selected for profit calculation.
3. The confidence interval used is 95%, taking 2.5% from both the upper and lower tails.
4. Retain only the regions with a loss risk lower than 2.5%. From the list of regions meeting the criteria, choose the region with the highest average profit.

[Back to Contents](#back)

### 5.1 Function to Calculate Profit <a id='bootstrap_function'></a>

In [40]:
sample_size = 500
bootstrap_size = 1000

In [41]:
def calculate_profit_bootstrap(prediction, name, income=4500, total_cost = 100000000, points = 200):
    predict_top200 = prediction.sort_values(ascending=False)[:points]
    product = predict_top200.sum()
    total_income = income * product
    profit = total_income - total_cost
    return profit

In [42]:
# function to calculate profit

def profit(target, predictions):
    predictions_sorted = predictions.sort_values(ascending=False)
    selected_points = target[predictions_sorted.index][:point_per_cost]
    product = selected_points.sum()
    revenue = product * product_price
    profit = revenue - total_cost
    return profit

### 5.2 Calculating profit in prediction data using bootstrapping technique <a id='bootstrap_prediction'></a>

In [43]:
for region in range(3):
    
    target = samples_target[region]
    predictions = samples_prediction[region]
    
    profit_values = []
    
    for i in tqdm(range(bootstrap_size)):
        target_sample= target.sample(n=sample_size, replace = True, random_state = state)
        prediction_sample= predictions[target_sample.index]
        profit_values.append(calculate_profit_bootstrap(prediction= prediction_sample, name=region))
        
    profit_values = pd.Series(profit_values)
    
    mean_profit = profit_values.mean()
    confidence_interval = (profit_values.quantile(0.025), profit_values.quantile(0.975))
    negative_profit_chance = (profit_values <0).mean()
    
    print('-- Region', region, '--')
    print('Mean Profit: ', mean_profit, 'USD')
    print('95% confidence interval:', confidence_interval)
    print('Risk of losses :', negative_profit_chance)
    print()

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:00<00:00, 1071.27it/s]


-- Region 0 --
Mean Profit:  3324499.1511043045 USD
95% confidence interval: (1010833.3330253005, 5593445.146503954)
Risk of losses : 0.002



100%|██████████| 1000/1000 [00:00<00:00, 1124.06it/s]


-- Region 1 --
Mean Profit:  4269058.93421017 USD
95% confidence interval: (348183.1922796317, 8115279.249680298)
Risk of losses : 0.016



100%|██████████| 1000/1000 [00:00<00:00, 1409.37it/s]

-- Region 2 --
Mean Profit:  3017845.5962736136 USD
95% confidence interval: (1167160.831588246, 5002914.4776899135)
Risk of losses : 0.001



**Analysis Results**

If using predicted data to calculate profit, the highest average profit is in region 1, although the risk of losses is also the highest among the three regions.

#### 5.3 Calculating profit in real data using bootstrapping technique <a id='bootstrap_real'></a>

In [44]:
for region in range(3):
    
    target = samples_target[region]
    predictions = samples_prediction[region]
    
    profit_values = []
    
    for i in tqdm(range(bootstrap_size)):
        target_sample= target.sample(n=sample_size, replace = True, random_state = state)
        prediction_sample= predictions[target_sample.index]
        profit_values.append(target_profit(target_sample, prediction_sample))
        
    profit_values = pd.Series(profit_values)
    
    mean_profit = profit_values.mean()
    confidence_interval = (profit_values.quantile(0.025), profit_values.quantile(0.975))
    negative_profit_chance = (profit_values <0).mean()
    
    print('-- Region', region, '--')
    print('Mean Profit: ', mean_profit, 'USD')
    print('95% confidence interval:', confidence_interval)
    print('Risk of losses :', negative_profit_chance)
    print()

100%|██████████| 1000/1000 [00:01<00:00, 580.02it/s]


-- Region 0 --
Mean Profit:  4341456.987107678 USD
95% confidence interval: (-576180.0642516082, 9645925.33771113)
Risk of losses : 0.049



100%|██████████| 1000/1000 [00:01<00:00, 716.02it/s]


-- Region 1 --
Mean Profit:  4841242.252647654 USD
95% confidence interval: (488526.54075440543, 8966394.898976061)
Risk of losses : 0.014



100%|██████████| 1000/1000 [00:01<00:00, 664.72it/s]


-- Region 2 --
Mean Profit:  3451884.409861117 USD
95% confidence interval: (-1898801.4969076396, 8989624.74503065)
Risk of losses : 0.116



**Analysis Results**

If using real/target data to calculate profit, the highest average profit is in region 1 with the smallest risk of losses, which is 0.014.

### 5.4 Comparison of Profit Values based on prediction and real data using bootstrapping technique <a id='bootstrap_predict_real_comparison'></a>

The highest average profit value is in region 0, both using predicted data and real data, although the level of risk of losses differs.

# General Conclusions <a id='end'></a>

Results from each process:

1. In the initial stage, accuracy calculation was performed on the predicted results of the Linear Regression model. Region 1 (37.75) had the lowest RMSE value (0.89), with a significant difference from regions 0 and 2 (40.23). This indicates that the actual average barrel value is between 67.81 (68.72 - 0.89) and 69.7 (68.72 + 0.89).
2. Profit calculation without bootstrapping technique, using predicted data, showed that region 0 had the highest profit value at 40M. However, when using real data (samples_target), the profit value for region 0 was 34M. This resulted in an error of about 6M in the predicted data.
3. The highest profit calculation using the bootstrapping technique, using predicted data, showed that region 1 had the highest average profit value. The highest average profit value using real data was also in region 1, with the lowest risk of losses (negative profit) value.

**Best Region Based on Analysis Results**

From the analysis results, **Region 1** is the best area for developing new oil wells. This is based on the small RMSE value of 0.89 and the low risk of losses value of 0.016, which is below the applied threshold (2.5%).

[Back to Contents](#back)